In [4]:
import os.path

import pandas as pd
import json

In [5]:
df_interactions = pd.read_csv("data/covid_twitter_sample_2020-04-16_interactions.csv")

In [6]:
with open("openai_key.json") as f:
    openai_key = json.load(f)["api_key"]

In [7]:
df_interactions.columns

Index(['created_date', 'topic', 'topic_prob', 'original_tweet_id',
       'original_tweet_text', 'original_user_id', 'reply_tweet_id',
       'reply_text', 'reply_user_id'],
      dtype='object')

## Usamos chatGPT para etiquetar

https://chatgpt.com/share/693ed840-e380-8013-b8ce-b9afd55645ac

## Generamos muestra para revision manual

In [8]:
df_labeled_chatgpt = pd.read_csv("data/covid_twitter_sample_2020-04-16_interactions_labeled_chatgpt.csv")
df_labeled_chatgpt.columns

Index(['created_date', 'topic', 'topic_prob', 'original_tweet_id',
       'original_tweet_text', 'original_user_id', 'reply_tweet_id',
       'reply_text', 'reply_user_id', 'label'],
      dtype='object')

In [12]:
df_labeled_chatgpt[['original_tweet_text', 'reply_text', 'label']].sample(150).to_csv("data/covid_twitter_sample_2020-04-16_interactions_labeled_chatgpt_sample150.csv", index=False)

Ideas para mejorar el prompt:
- hacerlo en Español
- pedir que justifique su respuesta
-
- tratar hashtags con cuidado
- prestar atención a emojis para determinar el sentimiento de cada tuit
- A veces el tuit original contiene un ataque (por ejemplo al gobierno). Si la respuesta refuerza ese ataque (por ejemplo, sumando más críticas, o insultando), la respuesta es un apoyo al tuit original de ataque.
- Si el tuit original contiene un hashtag de apoyo a una causa o medida, y la respuesta también, esto es un indicador de que el segundo tuit apoya al primero.
- Si la respuesta repite de forma irónica parte del tuit original, tratando de ridiculizarlo, es un ataque.

## Etiquetado revisado
Luego de la revisión, generamos un prompt mejorado y corremos el etiquetado de forma programática usando la API de OpenAI.

In [9]:
system_prompt = """
Clasifica cada par de tuits analizando la relación entre ellos y asignando una de tres etiquetas: "apoyo" (el segundo tuit apoya, está de acuerdo o desarrolla lo planteado en el primero), "ataque" (el segundo tuit critica, contradice o ridiculiza al primero) o "neutral" (el segundo tuit es irrelevante, no expresa una postura clara o mantiene la neutralidad respecto al primero).

Para cada par de tuits:
- Lee atentamente ambos tuits.
- Expón tu razonamiento paso a paso: ¿Hay evidencia de que el segundo tuit está de acuerdo o apoya al primero? ¿Lo ataca, critica u opone? ¿O no hay una postura clara o el contenido es irrelevante?
- tratar hashtags con cuidado
- prestar atención a emojis para determinar el sentimiento de cada tuit
- A veces el tuit original contiene un ataque (por ejemplo al gobierno). Si la respuesta refuerza ese ataque (por ejemplo, sumando más críticas, o insultando), la respuesta es un apoyo al tuit original de ataque.
- Si el tuit original contiene un hashtag de apoyo a una causa o medida, y la respuesta también, esto es un indicador de que el segundo tuit apoya al primero.
- Si la respuesta repite de forma irónica parte del tuit original, tratando de ridiculizarlo, es un ataque.
- Solo después de razonar minuciosamente, selecciona la etiqueta correcta.
- Repite el proceso para cada par proporcionado, hasta que todos hayan sido clasificados.

**Formato de salida:**
Para cada par de tuits, proporciona un objeto JSON con el siguiente formato:
{
  "tweet1": "[contenido del primer tuit]",
  "tweet2": "[contenido del segundo tuit]",
  "razonamiento": "[explicación paso a paso de cómo se determinó la relación]",
  "etiqueta": "apoyo" | "ataque" | "neutral"
}
Devuelve un arreglo con estos objetos JSON para múltiples pares.

**Ejemplo:**

Entrada:
Los tuits se presentarám como un arreglo en formato formato JSON, donde cada par de tuits es un objeto:
{
    "original_tweet_text": "[contenido del primer tuit]",
    "reply_text": "[contenido del segundo tuit]"
}

Salida:
[
  {
    "tweet1": "Creo que la salud universal es esencial para nuestro país.",
    "tweet2": "¡Totalmente de acuerdo! La salud debería estar al alcance de todos.",
    "razonamiento": "El segundo tuit expresa un acuerdo entusiasta con el primero al apoyar plenamente el acceso a la salud para todos, por lo que la relación es de apoyo.",
    "etiqueta": "apoyo"
  }
]

**Importante:**
- Siempre proporciona el **razonamiento antes de la etiqueta**.
- No empieces tu salida con la etiqueta.
- Sigue estrictamente este formato de salida.
- Si dudas sobre la relación, explica tu razonamiento detalladamente en el campo de razonamiento antes de decidir la etiqueta.

**Recordatorio:**
Revisa cada par de tuits, razona paso a paso sobre la relación y asigna la etiqueta *después* de completar tu razonamiento. Da la respuesta en el formato JSON especificado con explicaciones exhaustivas.

# Formato de salida

Para cada par de tuits proporciona un objeto JSON según el formato especificado arriba y reúne todos los pares en un arreglo. No incluyas ningún texto fuera del arreglo JSON de resultados.
"""

In [11]:
from openai import OpenAI
from openai.types.responses.response_output_message import ResponseOutputMessage

client = OpenAI(api_key=openai_key)

In [14]:
def label_interactions_with_llm(df_chunk):
    response = client.responses.create(
      model="gpt-5.2",
      input=[
        {
          "role": "developer",
          "content": [
            {
              "type": "input_text",
              "text": system_prompt
            }
          ]
        },
        {
            "role":    "user",
            "content": [
                {
                    "type": "input_text",
                    "text": json.dumps(
                        df_chunk[['original_tweet_text', 'reply_text']]
                        .to_dict(orient='records')
                    )
                }
            ]
        }
      ],
      text={
        "format": {
          "type": "text"
        },
        "verbosity": "medium"
      },
      reasoning={
        "effort": "medium",
        "summary": "auto"
      },
      tools=[],
      store=True,
      include=[
        "reasoning.encrypted_content",
        "web_search_call.action.sources"
      ]
    )
    response_output_message = [
        o for o in response.output if isinstance(o, ResponseOutputMessage)
    ][0]
    output_json = json.loads(
        response_output_message.content[0].text
    )

    return output_json

In [16]:
outputs_json_path = "data/covid_twitter_sample_2020-04-16_interactions_labeled_chatgpt_revised_full.json"

In [18]:
import os
if os.path.exists(outputs_json_path):
    with open(outputs_json_path) as f:
        outputs_json = json.load(f)
else:
    outputs_json = []

In [21]:
len(outputs_json)

790

In [22]:
for i in range(
        len(outputs_json), # start from where we left off
        len(df_interactions), 10):
    print(i)
    df_chunk = df_interactions[i:i+10]
    output_json_chunk = label_interactions_with_llm(df_chunk)
    outputs_json.extend(output_json_chunk)
    with open(outputs_json_path, "w") as f:
        json.dump(outputs_json, f, indent=2, ensure_ascii=False)    

790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960


In [26]:
len(outputs_json)

970

In [31]:
with open(outputs_json_path, 'w') as f:
    json.dump(outputs_json, f, indent=2, ensure_ascii=False)